# Defining dialects with PyRDL

The PyRDL Python DSL is the intended way to define operations.
It allows to define operations directly as Python code, without relying on generating code.
It generates the operation verifiers and accessors through the use of decorators,
and also allows to export the attribute and operations as IRDL programs.
It is also type-safe, which allows its use with typing tools such as `pyright`.

This notebook shows in details all of the different features of PyRDL.

## Defining attributes

Attributes are defined in PyRDL by subclassing either the `Data` or
`ParametrizedAttribute` class with a `irdl_attr_definition` decorator.
By convention, type class names are suffixed with `Type`, and attribute
with `Attr`.

`ParametrizedAttribute` is the preffered way to define attributes, though it is
sometimes necessary to use `Data`. The difference between the two classes is
that a `ParametrizedAttribute` attribute has a tuple of `Attribute` as
parameters, while a `Data` attribute as an arbitrary Python object as parameter.

Compared to MLIR, types are `Attribute`, and any attribute that also subclass
`TypeAttribute` is defining a type.

Here is an example of a pair type definition:

In [1]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.dialects.builtin import *

@irdl_attr_definition
class PairType(ParametrizedAttribute, TypeAttribute):
    name = "test.pair"

    left_type: ParameterDef[Attribute]
    right_type: ParameterDef[Attribute]

print(PairType([i32, i64]))

!test.pair<i32, i64>


### Attribute name

Each attribute definition should have a unique name that has the dialect
name as prefix. For example, `test.pair` is a valid name for a pair type
or attribute in the `test` dialect.

Names are defined by overriding the `name` class field. For example, the
`test.unit` attribute is defined as:

In [2]:
from xdsl.irdl import *
from xdsl.ir import *

@irdl_attr_definition
class UnitAttr(ParametrizedAttribute):
    """An attribute with no parameters."""
    name = "test.unit"

print(UnitAttr([]))

#test.unit


### Attribute parameters

Attribute parameters are defined differently for `Data` and `ParametrizedAttribute` definitions.

#### `ParametrizedAttribute` parameters

`ParametrizedAttribute` parameters are defined using the `ParameterDef` class.
The `ParameterDef` class takes as generic argument a PyRDL constraint, which
will check that the parameter is valid at the attribute creation. The parameters
can be accessed with the `parameter` field, or can be directly accessed using the 
field name.

Here is an example of a complex attribute with two `IntegerAttr` parameters.

In [55]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.dialects.builtin import *

@irdl_attr_definition
class ComplexAttr(ParametrizedAttribute):
    name = "test.complex"

    real: ParameterDef[IntegerAttr]
    imaginary: ParameterDef[IntegerAttr]

attr = ComplexAttr([IntegerAttr(4, i32), IntegerAttr(-2, i32)])
print("attribute:", attr)
print("real:", attr.real)
print("imaginary:", attr.imaginary)
print("number parameters:", len(attr.parameters))


attribute: #test.complex<4 : i32, -2 : i32>
real: 4 : i32
imaginary: -2 : i32
number parameters: 2


#### `Data` parameters

`Data` parameters are defined with the generic parameter of the `Data` class.
The parameter type should be a `python` class, and is allowed to be a generic class.
The parameter can be accessed with the `data` field.

For instance, here is the definition of a string data attribute:

In [56]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.parser import *
from xdsl.printer import *

T = TypeVar("T")

@irdl_attr_definition
class StringAttr(Data[list[T]]):
    name = "test.string"

    @classmethod
    def parse_parameter(cls, parser: AttrParser) -> str:
        parser.parse_punctuation("<")
        param = parser.parse_str_literal()
        parser.parse_punctuation(">")
        return param

    def print_parameter(self, printer: Printer) -> None:
        printer.print('<"', self.data , '">')

print("attribute:", StringAttr("hello"))
print("parameter:", StringAttr("hello").data)

attribute: #test.string<"hello">
parameter: hello


### Printing and Parsing

Parsing and printing methods are defined differently for `Data` and `ParametrizedAttribute` definitions.

#### `ParametrizedAttribute` attributes

`ParametrizedAttribute` have defaults parser/printer methods that is parsing
all parameters one by one, delimited by a comma. An attribute definition can
override these methods by defining `parse_parameters` and `print_parameters`
methods:

In [35]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.dialects.builtin import *

@irdl_attr_definition
class ComplexAttr(ParametrizedAttribute):
    name = "test.complex"

    real: ParameterDef[IntegerAttr]
    imaginary: ParameterDef[IntegerAttr]

    @classmethod
    def parse_parameters(cls, parser: AttrParser) -> list[Attribute]:
        parser.parse_punctuation("<")
        real = parser.parse_attribute()
        parser.parse_punctuation(",")
        imaginary = parser.parse_attribute()
        parser.parse_punctuation(">")
        return [real, imaginary]

    def print_parameters(self, printer: Printer) -> None:
        printer.print("<")
        printer.print_attribute(self.real)
        printer.print(", ")
        printer.print_attribute(self.imaginary)
        printer.print(">")

ctx = MLContext()
ctx.load_dialect(Builtin)
ctx.load_attr(ComplexAttr)

attr = Parser(ctx, "#test.complex<4 : i32, -2 : i32>").parse_attribute()
print(attr)

#test.complex<4 : i32, -2 : i32>


#### `Data` attributes

`Data` attributes are required to define printing and parsing methods,
as they rely on arbitrary Python objects. A `Data` definition uses the
`parse_parameter` and `print_parameter` methods to print and parse
its parameter:

In [36]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.dialects.builtin import *

@irdl_attr_definition
class StringAttr(Data[str]):
    name = "test.string"

    @classmethod
    def parse_parameter(cls, parser: AttrParser) -> str:
        parser.parse_punctuation("<")
        param = parser.parse_str_literal()
        parser.parse_punctuation(">")
        return param

    def print_parameter(self, printer: Printer) -> None:
        printer.print('<"', self.data , '">')

ctx = MLContext()
ctx.load_dialect(Builtin)
ctx.load_attr(StringAttr)

attr = Parser(ctx, '#test.string<"foobar">').parse_attribute()
print(attr)

#test.string<"foobar">


### Additional verifiers

`ParametrizedAttribute` and `Data` can optionally define a verifier that will
be called at the attribute creation time. It is defined with the `_verify`
method.

For `ParametrizedAttribute`s, this method is called after the PyRDL constraints
that each parameter has.

In [57]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.parser import *
from xdsl.printer import *

T = TypeVar("T")

@irdl_attr_definition
class EvenStringAttr(Data[list[T]]):
    """A string of even length"""
    name = "test.even_string"

    @classmethod
    def parse_parameter(cls, parser: AttrParser) -> str:
        parser.parse_punctuation("<")
        param = parser.parse_str_literal()
        parser.parse_punctuation(">")
        return param

    def print_parameter(self, printer: Printer) -> None:
        printer.print('<"', self.data , '">')

    def _verify(self) -> None:
        if len(self.data) % 2 != 0:
            raise VerifyException("String length must be even")

print(EvenStringAttr("even"))
try:
    EvenStringAttr("odd")
except VerifyException as e:
    print(e)

#test.even_string<"even">
String length must be even


### Custom constructors

By default, `Attribute`s have two default constructor. `new` will create a new attribute 
given either the parameter for `Data`, or the parameters for `ParametrizedAttribute`.
`new` should never be overriden, and custom constructors should override `__init__`
instead. `__init__` can be overriden by calling `super().__init__(...)`, which will
by default be equivalent to `new`:

In [4]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.dialects.builtin import *

@irdl_attr_definition
class ComplexAttr(ParametrizedAttribute):
    name = "test.complex"

    real: ParameterDef[IntegerAttr]
    imaginary: ParameterDef[IntegerAttr]

    def __init__(self, real: IntegerAttr, imaginary: IntegerAttr | None = None) -> None:
        if imaginary is None:
            imaginary = IntegerAttr(0, real.type)
        super().__init__([real, imaginary])

print(ComplexAttr.new([IntegerAttr(4, i32), IntegerAttr(0, i32)]))
print(ComplexAttr(IntegerAttr(4, i32), IntegerAttr(0, i32)))
print(ComplexAttr(IntegerAttr(4, i32)))

#test.complex<4 : i32, 0 : i32>
#test.complex<4 : i32, 0 : i32>
#test.complex<4 : i32, 0 : i32>


### Using generics in definitions

#### In `ParametrizedAttribute` definitions

Python generics (`TypeVar`) can also be used in `ParametrizedAttribute` PyRDL definitions.
This can be used to improve typing with `pyright` for instance, or can also be used in PyRDL constraints
for other attributes or operations.

The `bound` field of `TypeVar` is used in PyRDL for constraining of the parameter. As
attribute are immutable, the `covariant` parameter of the `TypeVar` should be set to
`True`.

When the generic parameters are specified in a PyRDL constraint, the constraint
will check that the parameters are correctly constrained:

In [4]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.dialects.builtin import *

SubT = TypeVar("SubT", bound=IntegerType | Float64Type, covariant=True)
"""A generic type that can either be an IntegerType or a Float64Type."""

@irdl_attr_definition
class ListType(Generic[SubT], ParametrizedAttribute, TypeAttribute):
    """
    A list type that can has either an IntegerType
    or a FloatType as parameters.
    """
    name = "test.pair"

    real: ParameterDef[SubT]

print(ListType([IntegerType(32)]))
print(ListType([Float64Type()]))
try:
    ListType([Float32Type()])
except VerifyException as e:
    print(f"Error while trying to build `!test.pair<f32>`: {e}")

@irdl_attr_definition
class CustomType(ParametrizedAttribute, TypeAttribute):
    """This type has as parameter a list of integers."""
    name = "test.custom"

    # Note that this parameter is specialized, and does not
    # accept all list types.
    parameter: ParameterDef[ListType[IntegerType]]

print(CustomType([ListType([IntegerType(32)])]))
try:
    CustomType([ListType([Float64Type()])])
except VerifyException as e:
    print(f"Error while trying to build `!test.custom<!test.pair<f32>>`: {e}")

!test.pair<i32>
!test.pair<f64>
Error while trying to build `!test.pair<f32>`: Unexpected attribute f32
!test.custom<!test.pair<i32>>
Error while trying to build `!test.custom<!test.pair<f32>>`: f64 should be of base attribute integer_type


#### Note on usage of generics in Python type hints

Note that when defining a generic type such as `PairType`, it is not possible
to use `PairType` directly as a type annotation, or in PyRDL constraints. For
that, it is necessary to specialize all of the type parameters (as of Python 3.12).
It is then useful to define `TypeAlias` for each generic type. Also, xDSL defines
the `isa` function that replace `isinstance` and can properly handle generic
attributes:

In [8]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.dialects.builtin import *
from xdsl.utils.hints import isa

SubT = TypeVar("SubT", bound=IntegerType | Float64Type, covariant=True)
"""A generic type that can either be an IntegerType or a Float64Type."""

@irdl_attr_definition
class ListType(Generic[SubT], ParametrizedAttribute, TypeAttribute):
    """
    A list type that can has either an IntegerType
    or a FloatType as parameters.
    """
    name = "test.pair"

    real: ParameterDef[SubT]

# This alias represent the type annotation for any possible list type.
AnyListType = ListType[IntegerType | Float64Type]

list_i32 = ListType([IntegerType(32)])
# assert isinstance(list_i32, ListType)      # This is not valid as of Python 3.12
# assert isinstance(list_i32, AnyListType)   # Nor is this
assert isa(list_i32, AnyListType)
assert isa(list_i32, ListType[IntegerType])
assert not isa(list_i32, ListType[Float64Type])

#### In `Data` definitions

As `Data` definitions handle arbitrary Python objects, the handling
of generic parameters cannot be automatically handled like with `ParametrizedAttribute`s.
`Data` with generic parameters, are instead defined with the `GenericData` class.
This class requires a `generic_constraint_coercion` method, that will be used
to define the proper PyRDL constraint when the generic attribute is being
specialized in a PyRDL constraint. It is also used in the `isa` function.

In [26]:
from xdsl.parser import *
from xdsl.printer import *
from xdsl.ir import *
from xdsl.irdl import *
from xdsl.utils.hints import isa
from xdsl.dialects.builtin import *
from dataclasses import dataclass

T = TypeVar("T", bound=Attribute, covariant=True)

@irdl_attr_definition
class ArrayAttr(GenericData[tuple[T, ...]]):
    name = "test.array"

    @staticmethod
    def generic_constraint_coercion(args: tuple[Any]) -> AttrConstraint:
        """
        This method is called when `ArrayAttr` is used in a PyRDL constraint
        or with `isa`.
        `args` is the tuple of type parameters passed to `ArrayAttr`. So here,
        it is a tuple of length 1.
        """
        return ArrayOfConstraint(irdl_to_attr_constraint(args[0]))

    @classmethod
    def parse_parameter(cls, parser: AttrParser) -> tuple[AttributeCovT, ...]:
        ...

    def print_parameter(self, printer: Printer) -> None:
        ...


@dataclass
class ArrayOfConstraint(AttrConstraint):
    """
    A constraint that enforces an ArrayData whose elements all satisfy
    `elem_constr`.
    To understand how `AttrConstraint` works, see the documentation on
    PyRDL constraints.
    """

    elem_constr: AttrConstraint

    def verify(self, attr: Attribute, constraint_vars: dict[str, Attribute]) -> None:
        if not isinstance(attr, ArrayAttr):
            raise VerifyException(f"expected ArrayData attribute, but got {attr}")
        for e in cast(ArrayAttr[Attribute], attr).data:
            self.elem_constr.verify(e, constraint_vars)

# These calls to isa are using the `generic_constraint_coercion` method
# defined above in `ArrayAttr`.
assert isa(ArrayAttr([IntegerAttr(4, i32)]), ArrayAttr[IntegerAttr])
assert isa(ArrayAttr([IntegerAttr(4, i32)]), ArrayAttr[Attribute])
assert not isa(ArrayAttr([IntegerAttr(4, i32)]), ArrayAttr[FloatAttr])

@irdl_attr_definition
class IntegerFunctionType(ParametrizedAttribute, TypeAttribute):
    name = "test.integer_function"

    # These parameter constraints are using the `generic_constraint_coercion`
    # method defined above in `ArrayAttr`.
    inputs: ParameterDef[ArrayAttr[IntegerType]]
    outputs: ParameterDef[ArrayAttr[IntegerType]]

IntegerFunctionType([ArrayAttr((i32, i32)), ArrayAttr((i64,))])
try:
    # This constructor call will fail because the second parameter is not
    # an array of integers.
    IntegerFunctionType([ArrayAttr((i32, i32)), ArrayAttr((f32,))])
except VerifyException as e:
    print(f"Error while trying to build `!test.integer_function<[i32, i32], [f32]>`: {e}")

Error while trying to build `!test.integer_function<[i32, i32], [f32]>`: f32 should be of base attribute integer_type



## Defining operations

### 

## Defining dialects

## PyRDL constraints

It allows to define operations directly as Python code, without relying on generating code. It is also type-safe, which allows its use with typing tools such as `pyright`.

This notebook shows in details all of the different features of PyRDL, but can also be used as a tutorial.

## Defining attributes

Data / ParametrizedAttribute

### Attribute name

name = ...

### Attribute parameters

#### Data parameters

#### Parametrized attr parameters

## Defining operations

### 

## Defining dialects

## PyRDL constraints